# My Pure Pix2Pix Implementation

****



## General Presets

In [16]:
""" Import Packages """

# Numerical Operations
import random
import numpy as np
# Reading/Writing/Cleaning Data
import pandas as pd
from PIL import Image
import os
import gc
# For Progress Bar
from tqdm.auto import tqdm
# For Display
import matplotlib.pyplot as plt
import wandb as wb
# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, random_split
from torchvision.datasets import DatasetFolder, VisionDataset
# my_utilities
import my_utilities


In [17]:
""" For Display """

plt.style.use(r'C:\Users\chael\.matplotlib\violet.mplstyle')
wb.init(project="pure_pix2pix", name="run0")


In [18]:
""" For Auto-reload Modules"""

%reload_ext autoreload
%autoreload 2

In [33]:
""" Assure Reproducibility """

seed=29
my_utilities.assure_reproduce(seed)

## Configurations

In [34]:
""" Path & Device"""

data_dir, checkpoints_dir = 'datasets/facades', 'checkpoints'
device=my_utilities.register_device()

In [52]:
""" Rule for loading """

# 'data_load_threads' =
# 'epoch_load' =
model_load = 'by iters' # | by_epochs

In [50]:
""" Model """

model_name = 'pix2pix'
translate_dirct = 'facades->photo' # | 'photo->facades',
hyper_params = {
    # preprocess
    'trans': 'scale & crop', # | crop | scale width | scale width & crop | none
    'scale_size': 286,
    'crop_size': 256,
    'flip': False, # whether to flip images in augmentation
    # network architectures
    'G_arch': 'U-net',
    'D_arch': 'PatchGAN', # (for experiments in the paper) | PixelGAN | StdCNN
    # 'G_layers':
    'D_layers': 3,
    'in_chan': 3,
    'out_chan': 3,
    'G_fil_lst': 64, # num. of filters in G's last conv. layer
    'D_fil_fst': 64, # num. of filters in D's first conv. layer
    # training """
    # 'init_type':
    'init_scaler': 0.02, # scaling factor for weight initialization
    'shuffle_batch': True,
    'batch_size': 16,
    'norm_type': 'instance', # (for experiments in the paper) | batch | none
    'G_dropout': False, # no dropout for G in the paper
    'D_dropout': True,
}

wb.config.update({'translate_dirct': translate_dirct, **hyper_params})

In [53]:
my_utilities.print_config(device, model_load, model_name, translate_dirct, hyper_params)

============CONFIGURATIONS============
Device = cuda
Model is loaded by iters
Model = pix2pix
Translation Dirct. = facades->photo
--------------------------------------
Transforms Opt. = scale & crop
Img. Scale Size = 286
Img. Crop Size = 256
Flipped = False
--------------------------------------
G's Arch. = U-net
D's Arch. = PatchGAN
num. of D's layers = 3
num. of Input's Channels = 3
num. of Output's Channels = 3
--------------------------------------
Batch Size = 16
Normalization = instance
G's Dropout = False
D's Dropout = True
